In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from data_utils import get_lazy_dataloaders
from model_utils import evaluate_model
from eeg_clip_basic import EEGToCLIPNet
from scipy.spatial.distance import correlation

from diffusers import StableUnCLIPImg2ImgPipeline
import os
import torch
import numpy as np

sub = 1
recon_dir = f"results/thingseeg2_preproc/sub-{sub:02d}/unclip/" # Directory to save the reconstructed images
os.makedirs(recon_dir, exist_ok=True)

# Start the StableUnCLIP Image variations pipeline
pipe = StableUnCLIPImg2ImgPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-1-unclip", torch_dtype=torch.float16, variation="fp16"
)

device = "cuda"
# model_id_or_path = "stable-diffusion-v1-5/stable-diffusion-v1-5"
# pipe = StableDiffusionImg2ImgPipeline.from_pretrained(model_id_or_path, torch_dtype=torch.float16)
pipe = pipe.to(device)

/home/ruichu-xia/workspace/neural-decoder/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Keyword arguments {'variation': 'fp16'} are not expected by StableUnCLIPImg2ImgPipeline and will be ignored.
Loading pipeline components...: 100%|██████████| 9/9 [00:05<00:00,  1.56it/s]


In [2]:
clip_train_loader, clip_val_loader, clip_test_loader = get_lazy_dataloaders(sub_id=1, batch_size=32, shuffle=True, embedding_type='clip')

for data in clip_val_loader:
    print(data[0].shape, data[1].shape)
    break

vae_train_loader, vae_val_loader, vae_test_loader = get_lazy_dataloaders(sub_id=1, batch_size=32, shuffle=True, embedding_type='vae')

for data in vae_val_loader:
    print(data[0].shape, data[1].shape)
    break

torch.Size([32, 1360]) torch.Size([32, 1024])
torch.Size([32, 1360]) torch.Size([32, 36864])


In [3]:
def get_numpy_data(loader):
    """Helper function to convert a PyTorch DataLoader to flattened NumPy arrays."""
    all_eeg = []
    all_embeddings = []
    for eeg_batch, embedding_batch in loader:
        all_eeg.append(eeg_batch.numpy())
        all_embeddings.append(embedding_batch.numpy())
    
    eeg_np = np.concatenate(all_eeg, axis=0)
    eeg_np_flat = eeg_np.reshape(eeg_np.shape[0], -1)
    embeddings_np = np.concatenate(all_embeddings, axis=0)
    
    return eeg_np_flat, embeddings_np

eeg_clip_train_np, clip_train_np = get_numpy_data(clip_train_loader)
eeg_clip_test_np, clip_test_np = get_numpy_data(clip_test_loader)
eeg_vae_train_np, vae_train_np = get_numpy_data(vae_train_loader)
eeg_vae_test_np, vae_test_np = get_numpy_data(vae_test_loader)

print(eeg_clip_train_np.shape, clip_train_np.shape)
print(eeg_clip_test_np.shape, clip_test_np.shape)
print(eeg_vae_train_np.shape, vae_train_np.shape)
print(eeg_vae_test_np.shape, vae_test_np.shape)

(13232, 1360) (13232, 1024)
(200, 1360) (200, 1024)
(13232, 1360) (13232, 36864)
(200, 1360) (200, 36864)


In [4]:
print("--- Benchmarking Ridge Regression Model (EEG -> Embedding) ---")

alpha = 100.0

print("Training Regression For CLIP...")
# This is the corrected model: predicting latents from eeg
clip_reg = Ridge(alpha=alpha, max_iter=50000, fit_intercept=True)
clip_reg.fit(eeg_clip_train_np, clip_train_np)
print('Training complete.')

clip_embedding_pred_ridge = clip_reg.predict(eeg_clip_test_np)
mse_clip_ridge = mean_squared_error(clip_test_np, clip_embedding_pred_ridge)
print(f"MSE (EEG -> CLIP): {mse_clip_ridge:.4f}\n")

--- Benchmarking Ridge Regression Model (EEG -> Embedding) ---
Training Regression For CLIP...
Training complete.
MSE (EEG -> CLIP): 0.2781



In [5]:
print("Training Regression For VAE...")
vae_reg = Ridge(alpha=alpha, max_iter=50000, fit_intercept=True)
vae_reg.fit(eeg_vae_train_np, vae_train_np)
print('Training complete.')

vae_embedding_pred_ridge = vae_reg.predict(eeg_vae_test_np)
mse_vae_ridge = mean_squared_error(vae_test_np, vae_embedding_pred_ridge)
print(f"MSE (EEG -> VAE): {mse_vae_ridge:.4f}\n")

Training Regression For VAE...
Training complete.
MSE (EEG -> VAE): 14.6775



In [28]:
def evaluate_reg(reg, embeddings_train_np,eeg_test_np, embeddings_test_np):
    pred_latent = reg.predict(eeg_test_np)
    pred_latent_mean = np.mean(pred_latent,axis=0)
    pred_latent_std = np.std(pred_latent,axis=0)
    std_norm_pred_latent = (pred_latent - pred_latent_mean) / pred_latent_std
    train_latents_mean = np.mean(embeddings_train_np,axis=0)
    train_latents_std = np.std(embeddings_train_np,axis=0)
    pred_latents = std_norm_pred_latent * train_latents_std + train_latents_mean

    # np.save(save_dir + latent_filename, pred_latents)

    # Compute the Euclidean distances
    euclidean_distances = np.array([np.linalg.norm(u - v) for u, v in zip(pred_latents, embeddings_test_np)])
    correlation_distances = np.array([correlation(u, v) for u, v in zip(pred_latents, embeddings_test_np)])
    # Compute the average Euclidean distance
    average_euclidean_distance = euclidean_distances.mean()
    correlations = (1 - correlation_distances).mean()
    print(reg.score(eeg_test_np,embeddings_test_np), average_euclidean_distance, correlations) 

In [29]:
evaluate_reg(clip_reg, clip_train_np, eeg_clip_test_np, clip_test_np)

0.03358376771211624 21.325577 0.5347709


In [12]:
evaluate_reg(vae_reg, vae_train_np, eeg_vae_test_np, vae_test_np)

0.06044551357626915 905.81464 0.5810456


In [14]:
sub_id = 1
eeg_dir = 'data/thingseeg2_preproc/'
embedding_dir = 'cache/thingseeg2_extracted_embeddings/'
eeg_test_path = f"{eeg_dir}sub-{sub_id:02d}/test_thingseeg2_avg.npy"
clip_train_path = f"{embedding_dir}train_clip.npy"
clip_test_path = f"{embedding_dir}test_clip.npy"
vae_train_path = f"{embedding_dir}train_vae.npy"
vae_test_path = f"{embedding_dir}test_vae.npy"

eeg_test = np.load(eeg_test_path).reshape(200, -1)
clip_train = np.load(clip_train_path)
clip_test = np.load(clip_test_path)
vae_test = np.load(vae_test_path)
vae_train = np.load(vae_train_path)

print(eeg_test.shape, clip_test.shape, vae_test.shape)

(200, 1360) (200, 1024) (200, 36864)


In [30]:
np.load(eeg_test_path).shape

(200, 17, 80)

In [15]:
def make_predictions(reg, eeg_test, train_latents):
    pred_latent = reg.predict(eeg_test)
    pred_latent_mean = np.mean(pred_latent,axis=0)
    pred_latent_std = np.std(pred_latent,axis=0)
    std_norm_pred_latent = (pred_latent - pred_latent_mean) / pred_latent_std
    train_latents_mean = np.mean(train_latents,axis=0)
    train_latents_std = np.std(train_latents,axis=0)
    pred_latents = std_norm_pred_latent * train_latents_std + train_latents_mean
    return pred_latents

In [16]:
pred_clip = make_predictions(clip_reg, eeg_test, clip_train)
pred_vae = make_predictions(vae_reg, eeg_test, vae_train)

print(pred_clip.shape, pred_vae.shape)

(200, 1024) (200, 36864)


In [17]:
pred_clip_path = "cache/pred_clip.npy"
pred_vae_path = "cache/pred_vae.npy"
np.save(pred_clip_path, pred_clip)
np.save(pred_vae_path, pred_vae)

In [18]:
sub = 1
recon_dir = f"results/thingseeg2_preproc/sub-{sub:02d}/unclip/" # Directory to save the reconstructed images
os.makedirs(recon_dir, exist_ok=True)
device = 'cuda'
#Start the StableUnCLIP Image variations pipeline
pipe = StableUnCLIPImg2ImgPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-1-unclip", torch_dtype=torch.float16, variation="fp16"
).to(device)

Keyword arguments {'variation': 'fp16'} are not expected by StableUnCLIPImg2ImgPipeline and will be ignored.
Loading pipeline components...: 100%|██████████| 9/9 [00:01<00:00,  8.30it/s]


In [24]:

for i, embedding in enumerate(pred_clip):
    print(i)
    vae_latent = pred_vae[i].reshape((1, 4, 96, 96))
    vae_latent = torch.from_numpy(vae_latent).to(device).half()
    torch.manual_seed(0)
    noise_latent=torch.randn(vae_latent.shape, device=device).half()
    vae_latent = vae_latent*0.02 + noise_latent
    embedding = torch.tensor(embedding, device=device, dtype=torch.float16).unsqueeze(0)
    image = pipe(image_embeds=embedding, latents=vae_latent, guidance_scale=7.5).images[0]
    image.save(recon_dir + f"{i}.png")

0


  0%|          | 0/21 [00:00<?, ?it/s]

100%|██████████| 21/21 [00:01<00:00, 15.83it/s]


1


100%|██████████| 21/21 [00:01<00:00, 19.32it/s]


...

100%|██████████| 21/21 [00:01<00:00, 19.23it/s]


198


100%|██████████| 21/21 [00:01<00:00, 19.48it/s]


199


100%|██████████| 21/21 [00:01<00:00, 19.44it/s]
